## 실제 사이트 크롤링 - 네이버 환율

In [1]:
import bs4
import requests as rq
import pandas as pd

### 1. 사이트에 접속하고 탐색해 본다:

1. 먼저 다음 사이트에 들어가서 살펴본다:  [네이버 금융](https://finance.naver.com/marketindex/)
2. 크롬에서 3점 메뉴 버튼을 누르고 **도구 더보기** ==> **개발자 도구**를 선택해서 개발자 도구를 연다.
3. 개발자 도구 왼쪽 상단의 화살표 버튼을 눌러서 사이트 일부를 선택하고 해당 HTML코드를 볼 수 있다. "[CTRL] + [SHIFT] + C" 조합으로 대체할 수 있다.

In [2]:
# 웹사이트의 URL에 접속.
res = rq.get("https://finance.naver.com/marketindex")   #res=rq.get("주소"): 데이터가져올 사이트 주소

In [3]:
# status_code가 200이면 OK.
# status_code가 4xx이면 접속 오류.
res.status_code

200

In [4]:
print(res.text)     # 데이터 전체를 출력(내용을 텍스트로 보여줌)-아직은 파싱이 필요한 상태
                    # print(res.content)와 비교.              









<script language=javascript src="/template/head_js.nhn?referer=info.finance.naver.com&menu=marketindex&submenu=market"></script>
<script type="text/javascript" src="/js/info/jindo.min.ns.1.5.3.euckr.js"></script>
<script type="text/javascript" src="/js/jindo.1.5.3.element-text-patch.js"></script>


	
	
	<div id="container" style="padding-bottom:0px;">
	

		




<script language="JavaScript" src="/js/flashObject.js?20200107172122"></script>

























		<div class="market_include">
	<div class="market_data">
		<div class="market1">
			<div class="title">
				<h2 class="h_market1"><span>환전 고시 환율</span></h2>

			</div>
			<!-- data -->
			<div class="data">
				
				<ul class="data_lst" id="exchangeList">
				
				    
				         
				         
				         
				         				         			         
				        
					
				 	<li class="on">
					<a href="/marketindex/exchangeDetail.nhn?marketindexCd=FX_USDKRW" class="head usd" onClick="clickcr(this, 'fr1.usdt', '', '',

In [5]:
print(res.content)         # 데이터 전체를 출력(내용을 텍스트로 보여줌)-아직은 파싱이 필요한 상태

b'\n\n\n\n\n\n\n\n<script language=javascript src="/template/head_js.nhn?referer=info.finance.naver.com&menu=marketindex&submenu=market"></script>\n<script type="text/javascript" src="/js/info/jindo.min.ns.1.5.3.euckr.js"></script>\n<script type="text/javascript" src="/js/jindo.1.5.3.element-text-patch.js"></script>\n\n\n\t\n\t\n\t<div id="container" style="padding-bottom:0px;">\n\t\n\n\t\t\n\n\n\n\n<script language="JavaScript" src="/js/flashObject.js?20200107172122"></script>\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\t<div class="market_include">\n\t<div class="market_data">\n\t\t<div class="market1">\n\t\t\t<div class="title">\n\t\t\t\t<h2 class="h_market1"><span>\xc8\xaf\xc0\xfc \xb0\xed\xbd\xc3 \xc8\xaf\xc0\xb2</span></h2>\n\n\t\t\t</div>\n\t\t\t<!-- data -->\n\t\t\t<div class="data">\n\t\t\t\t\n\t\t\t\t<ul class="data_lst" id="exchangeList">\n\t\t\t\t\n\t\t\t\t    \n\t\t\t\t         \n\t\t\t\t         \n\t\t\t\t         \n\t\t\t\t         \t\t\t\t         \t\t\t      

### 2. 파싱

In [6]:
my_html = res.text      #res.text를 변수 my_html에 입력
soup = bs4.BeautifulSoup(my_html, 'html.parser')        #soup = bs4.BeautifulSoup(res.text,'html.parser') 
                                                        #html을 해석해주는 parser3가지 중 우리는 'html.parser'사용
print(soup.prettify())                                  #BeautifulSoup 객체 반환. 

<script language="javascript" src="/template/head_js.nhn?referer=info.finance.naver.com&amp;menu=marketindex&amp;submenu=market">
</script>
<script src="/js/info/jindo.min.ns.1.5.3.euckr.js" type="text/javascript">
</script>
<script src="/js/jindo.1.5.3.element-text-patch.js" type="text/javascript">
</script>
<div id="container" style="padding-bottom:0px;">
 <script language="JavaScript" src="/js/flashObject.js?20200107172122">
 </script>
 <div class="market_include">
  <div class="market_data">
   <div class="market1">
    <div class="title">
     <h2 class="h_market1">
      <span>
       환전 고시 환율
      </span>
     </h2>
    </div>
    <!-- data -->
    <div class="data">
     <ul class="data_lst" id="exchangeList">
      <li class="on">
       <a class="head usd" href="/marketindex/exchangeDetail.nhn?marketindexCd=FX_USDKRW" onclick="clickcr(this, 'fr1.usdt', '', '', event);">
        <h3 class="h_lst">
         <span class="blind">
          미국 USD
         </span>
        </h3>
 

### 3. 환율 가져오기: 2가지 방법


#### 1) find_all 메서드 사용한 방법

In [7]:
res = soup.find_all('span', class_='value')                # span 태그 중 클래스가 "value" 인 것
rate =[]                                                   #가져옴
for i in range(4):                                         #첫 4개만 가져옴
    rate += [float(''.join((res[i].text).split(',')))]     #1000 단위 쉼표 "," 제거 처리.
                                                          #문자열strig으로 주어짐 1000단위는 쉼표가 있음
                                                    #자료를 정리하고 싶어 join((res[i].text).split(',')))
                                                    #res에서 i번째 요소가져와 .text하니 금액이 문자열로 나옴
                                                    #문자열을 콤마기준으로 쪼갬 - 문자열.split(',')
                                                    #콤마는 숫자가 아니여서 숫자변환시 문제가 됨
                                                    #이것을 join함수의 인자값으로 넣고
                                                    #그 앞에는 빈 string을 넣음
                                                    #그 다음 float하고 리스트에 넣음
        
result = {"외환":["USD","JPY","EUR","CNY"], "환율":rate}   #외환키에 외환리스트, 환율키에 환율리스트 넣어줌
df=pd.DataFrame(result)                                    #데이터프레임으로 구조화함
df                                                

,외환,환율
0,USD,1159.00
1,JPY,1052.73
2,EUR,1288.11
3,CNY,168.85


In [8]:
df.info()     #외환은 object, 환율은 float이라 이제 연산에 사용가능

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
외환    4 non-null object
환율    4 non-null float64
dtypes: float64(1), object(1)
memory usage: 192.0+ bytes


#### 2) select 메서드 사용한 방법

In [9]:
res = soup.select('span.value')                             # span 태그 중 "value" 패턴 있는 클래스.   #select에선 이부분만 다름
rate =[]
for i in range(4):
    rate += [float(''.join((res[i].text).split(',')))]     # 1000 단위 쉼표 "," 제거 처리.
result = {"외환":["USD","JPY","EUR","CNY"], "환율":rate}
df=pd.DataFrame(result)
df

,외환,환율
0,USD,1159.00
1,JPY,1052.73
2,EUR,1288.11
3,CNY,168.85


### 4. extract 사용해서 요소 지우기:

In [10]:
print(soup.find('a').prettify())     # soup.find('a"): 첫 번째 a태그 보기

<a class="head usd" href="/marketindex/exchangeDetail.nhn?marketindexCd=FX_USDKRW" onclick="clickcr(this, 'fr1.usdt', '', '', event);">
 <h3 class="h_lst">
  <span class="blind">
   미국 USD
  </span>
 </h3>
 <div class="head_info point_dn">
  <span class="value">
   1,159.00
  </span>
  <span class="txt_krw">
   <span class="blind">
    원
   </span>
  </span>
  <span class="change">
   1.50
  </span>
  <span class="blind">
   하락
  </span>
 </div>
</a>



In [11]:
soup.find('a').extract()             # 첫 번째 a태그 지우기 extract()

<a class="head usd" href="/marketindex/exchangeDetail.nhn?marketindexCd=FX_USDKRW" onclick="clickcr(this, 'fr1.usdt', '', '', event);">
<h3 class="h_lst"><span class="blind">미국 USD</span></h3>
<div class="head_info point_dn">
<span class="value">1,159.00</span>
<span class="txt_krw"><span class="blind">원</span></span>
<span class="change"> 1.50</span>
<span class="blind">하락</span>
</div>
</a>

In [12]:
print(soup.find('a').prettify())    # 첫 번째 다음의 a태그 보기

<a class="graph_img" href="/marketindex/exchangeDetail.nhn?marketindexCd=FX_USDKRW" onclick="clickcr(this, 'fr1.usdc', '', '', event);">
 <img alt="" height="153" src="https://ssl.pstatic.net/imgfinance/chart/marketindex/FX_USDKRW.png" width="295"/>
</a>

